# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
city_data = pd.read_csv("output_data/city_data.csv")
city_data.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,0,Bredasdorp,-34.53,20.04,51.80,87,5,12.75
1,1,Brae,60.40,-1.35,55.40,76,20,10.29
2,2,San Quintín,30.48,-115.95,68.20,80,0,2.44
3,3,Nioro,13.35,-15.75,86.11,56,100,6.73
4,4,Daru,-9.08,143.21,77.49,87,100,17.69


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
#Configure gmaps.
gmaps.configure(api_key=g_key)

#Store Latitude and Longitude in Locations
locations = city_data[["Latitude", "Longitude"]].astype(float)

#Store Humidity in Humidity
humidity = city_data["Humidity"].astype(float)   

In [14]:
#Plot Heatmap
fig = gmaps.figure()

#Create Heat Layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
#Add Heat Layer
fig.add_layer(heat_layer)

#Display
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
#Narrow down the cities to fit weather conditions.
ideal_cities = city_data.loc[(city_data["Temperature"] > 70) & (city_data["Temperature"] < 80) & (city_data["Cloudiness"] == 0) & (city_data["Wind Speed"] < 10), :]
ideal_cities.reset_index(inplace=True)
del ideal_cities['index']

#Drop any rows will null values.
ideal_cities = ideal_cities.dropna(how='any')
ideal_cities

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,36,Tyumentsevo,53.32,81.50,70.14,78,0,5.08
1,145,Nizhniy Kuranakh,58.84,125.49,74.37,69,0,3.58
2,152,Saint-François,46.42,3.91,75.54,35,0,5.82
3,186,Rio Brilhante,-21.80,-54.55,75.45,53,0,5.28
4,195,Jardim,-21.48,-56.14,75.99,53,0,7.18
5,209,Tambovka,50.10,128.05,77.00,69,0,6.71
6,268,Lipetsk,52.60,39.57,78.80,39,0,4.47
7,290,Cabo San Lucas,22.89,-109.91,79.00,66,0,5.95
8,316,Konstantinovka,49.62,127.98,71.17,82,0,3.22
9,321,Kirsanov,52.65,42.72,75.52,51,0,6.98


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
#Create list to store hotel names
hotellist = []

for i in range(len(ideal_cities)):
    lat = ideal_cities.loc[i]['Latitude']
    lng = ideal_cities.loc[i]['Longitude']
    
#Set parameters to search for hotels with 5000 meters.
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
#Hit the Google Places API for each city's coordinates.    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    
#Store the first Hotel result in the list     
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")

#Update the dataframe with Hotel Name column and populate from list       
ideal_cities["Hotel Name"] = hotellist

ideal_cities

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,36,Tyumentsevo,53.32,81.50,70.14,78,0,5.08,Tyumentsevo
1,145,Nizhniy Kuranakh,58.84,125.49,74.37,69,0,3.58,Nizhny Kuranakh
2,152,Saint-François,46.42,3.91,75.54,35,0,5.82,Molinet
3,186,Rio Brilhante,-21.80,-54.55,75.45,53,0,5.28,Rio Brilhante
4,195,Jardim,-21.48,-56.14,75.99,53,0,7.18,Jardim
5,209,Tambovka,50.10,128.05,77.00,69,0,6.71,Tambovka
6,268,Lipetsk,52.60,39.57,78.80,39,0,4.47,Lipetsk
7,290,Cabo San Lucas,22.89,-109.91,79.00,66,0,5.95,Cabo San Lucas
8,316,Konstantinovka,49.62,127.98,71.17,82,0,3.22,Heihe
9,321,Kirsanov,52.65,42.72,75.52,51,0,6.98,Kirsanov


In [18]:
#verifying JSON data since Hotel name and City name were at times the same name.
jsn


{'html_attributions': [],
 'next_page_token': 'CqQCGgEAAC6X1BoptEl9wnz71qhdAIMjShTy4UEaSzKAuAzjDgFTdlDhXkPv6rj4FcxPCEp1OmTGoq9YucyU-f-ed_2jgV6QdV7_-gofIVgM2bVYULIYk4qUcLDLhtW8towiTr6e0FwHwPs-OoZzukRfU2LAtslb60hMIunVNB26Fd_9pgdqEDzv7w-2Rs-nHUad-PNcDnGVsdIMS6FHu5tNSaxXs7dP-DArUIL4q4aGiUYug-3o9TLPKqksMqnn8zht42xiFiwbp5cRaQZu8xwnqhuH_8um7Zrk0AToKWk1yJSwA4jeP51rUx0fyCPyHEui5JbGXVE9iPWxNgCMUklqDTyfuN01_7okDbJz3zMcuqEczU8B79-sots9EVsDQsN9OQkPbBIQSlrW530FjfC7osd1HuzuJBoUNAucIl4U8O5mEytPfq15MnyIWg8',
 'results': [{'geometry': {'location': {'lat': 51.1828188,
     'lng': 32.83299170000001},
    'viewport': {'northeast': {'lat': 51.20258399999999, 'lng': 32.8753651},
     'southwest': {'lat': 51.1485019, 'lng': 32.7789631}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/geocode-71.png',
   'id': '7b44df67e2eb8ff12074f85f7db0e4f9c9202f64',
   'name': 'Bakhmach',
   'photos': [{'height': 480,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/1086459751077124

In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_cities.iterrows()]
locations_list = ideal_cities[["Latitude", "Longitude"]]

In [20]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations_list, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))